______
<img style="float: right;" src="./images/headerlogo.png">
## Google Earth Engine Tutorial
### Linear filters
_____





It is not uncommon to apply filters to enhance images for visualization or previous step for other image processing. Popular methods are convolution filters. These methods calculate a new image applying a moving window over the original pixels contained in the defined window. The following image adapted from [Richards, J. (2013)](http://www.springer.com/gp/book/9783642300615), illustrates the operation of a moving window.<br>
![alt text](./images/filter_window.png "Filter window over an image")

In GEE the term *kernel* is used interchangeable with *kernel*. Then, to apply a filter, it is necessary to define the weigths and size of the moving window.  After defining the filter, a *convolve* function is used to apply it. 
GEE includes several filters or kernels. In this example,
we will select a square kernel that has the following parameters: <br>
+ *radius (Float)* :  The radius of the kernel to generate. For a 3x3 kernel, we should use 1, for 5x5 kernel we should use 2 and so on. <br>
+ *units (String) *:The system of measurement for the kernel "meters" or "pixels". Default value: "pixels".<br>
+ *normalize (Boolean)*: Indicates if the kernel values should be normalize to sum 1. Default value: "True".<br>
+ *magnitude (Float) *: Scale each value by this amount. Default: 1.<br>
The kernel definition of a mean filter looks like the image below: <br> 
![alt text](./images/filter_mean.png "Kernel definition")
This filter will smooth the image as can be seen after applying the following code.

In [1]:
import ee
from IPython.display import Image
ee.Initialize()
# load our image 
visimage = ee.Image("users/rosamaguilar/tutorial/subset")
# Rename the bands for convenience 
visimage = visimage.select(
   ['b5','b3','b2'],   # current names
    ['Red', 'Green', 'Blue']  # new names
)
# visualize the original image
Image(url=visimage.getThumbUrl({'min': 0, 'max': 2048,'bands': 'Red,Green,Blue'}))

In [2]:
# defining the filter
lpk = ee.Kernel.square(radius = 1, units = 'pixels', normalize= True)  # filter mean of 3*3 
# apply the filter using the function convolve
smooth3x3 =    visimage.convolve(lpk)
# visualize the result
Image(url=smooth3x3.getThumbUrl({'min': 0, 'max': 2048,'bands': 'Red,Green,Blue'}))

```python
# Increasing the size of the moving window. 
# defining the filter
lpk = ee.Kernel.square(radius = 3, units = 'pixels', normalize= True)  # filtro de 7*7 
# apply the filter using the function convolve
smooth7x7 =    visimage.convolve(lpk)
# visualize the result
Image(url=smooth7x7.getThumbUrl({'min': 0, 'max': 2048,'bands': 'Red,Green,Blue'}))
```

Several predefined kernel are available in GEE. Circle, rectangle, Gaussian, among others. <br>
We can also define the kernel using the <i>fixed</i> kernel option. <br>
Parameters for a fixed kernel are: <br>
+ *width (Integer)*: The width of the kernel in pixels.
+ *height (Integer)*: The height of the kernel in pixels.
+ *weights (List)*: The pixel values of the kernel.
+ *x (Integer, default: -1)*:
The location of the focus, as an offset from the left.
+ *y (Integer, default: -1)*:The location of the focus, as an offset from the top.
+ *normalize (Boolean, default: false)*:Normalize the kernel values to sum to 1.

```python
# Lets apply a edge detection filter of 1x3.
# define the kernel
list =  [[-1,0 , 1]]
kernel = ee.Kernel.fixed(3, 1, list, -1, 0, False)
print(kernel.getInfo())
```

In [5]:
# apply the kernel
edge1x3 = visimage.convolve(kernel)

# visualize the result
Image(url=edge1x3.getThumbUrl({'min': 0, 'max': 2048,'bands': 'Red,Green,Blue'}))

In [6]:
# Lets apply a edge detection filter of 3x1.
# define the kernel
list =  [[-1],[0] , [1]]
kernel = ee.Kernel.fixed(1, 3, list, -1, 0, False)
print(kernel.getInfo())
# apply the kernel
edge1x3 = visimage.convolve(kernel)

# visualize the result
Image(url=edge1x3.getThumbUrl({'min': 0, 'max': 2048,'bands': 'Red,Green,Blue'}))

{'x': 0, 'height': 3, 'weights': '\n  [-1.0]\n  [0.0]\n  [1.0]', 'center': [0, 0], 'type': 'Kernel.fixed', 'width': 1, 'y': 0}


It maybe of interest to apply a [laplacian filter](https://en.wikipedia.org/wiki/Discrete_Laplace_operator).  <br>
 A possible kernel for a laplacian filter is: <br>
![alt text](./images/laplacian.png "Laplacian Kernel")

In [7]:
# define the kernel for laplacian filter
import pprint
list =  [0,1,0]
center  = [1,-4,1]
klist = [list,center,list]
kernel = ee.Kernel.fixed(3, 3, klist, -1, -1, False)

pprint.pprint(kernel.getInfo())
# apply the kernel
laplacian = visimage.convolve(kernel)

# visualize the result

Image(url=laplacian.getThumbUrl({'min': 0, 'max': 2048,'bands': 'Red,Green,Blue'}))

{'center': [1, 1],
 'height': 3,
 'type': 'Kernel.fixed',
 'weights': '\n  [0.0, 1.0, 0.0]\n  [1.0, -4.0, 1.0]\n  [0.0, 1.0, 0.0]',
 'width': 3,
 'x': 1,
 'y': 1}


In [8]:
# Lets try another example, using a gaussian filter
# define the kernel, the code below will produce a discrete aproximation of a gaussian filter with sigma= 1 and size=5x5 <br> 
gauskernel = ee.Kernel.gaussian(2,1)
pprint.pprint(gauskernel.getInfo())
# apply the filter
gauss=  visimage.convolve(gauskernel)
# visualize results 
Image(url=gauss.getThumbUrl({'min': 0, 'max': 2048,'bands': 'Red,Green,Blue'}))

{'center': [2, 2],
 'radius': 2.0,
 'type': 'Kernel.gaussian',
 'weights': '\n'
            '  [0.0029690167439504977, 0.013306209891013656, '
            '0.021938231279714653, 0.013306209891013656, '
            '0.0029690167439504977]\n'
            '  [0.013306209891013656, 0.05963429543618016, 0.0983203313488458, '
            '0.05963429543618016, 0.013306209891013656]\n'
            '  [0.021938231279714653, 0.0983203313488458, 0.1621028216371267, '
            '0.0983203313488458, 0.021938231279714653]\n'
            '  [0.013306209891013656, 0.05963429543618016, 0.0983203313488458, '
            '0.05963429543618016, 0.013306209891013656]\n'
            '  [0.0029690167439504977, 0.013306209891013656, '
            '0.021938231279714653, 0.013306209891013656, '
            '0.0029690167439504977]'}


We have defined a particular laplacian kernel. But, it is also possible to use some laplacian kernels that  GEE has already defined.  The follwing code illustrates a 

In [9]:
# define the kernel <br> 
kernel = ee.Kernel.laplacian8()
pprint.pprint(kernel.getInfo())
# apply the filter
laplacian8 =  visimage.convolve(kernel)
# visualize results 
Image(url = laplacian8.getThumbUrl({'min': 0, 'max': 2048,'bands': 'Red,Green,Blue'}))

{'center': [1, 1],
 'type': 'Kernel.laplacian8',
 'weights': '\n  [1.0, 1.0, 1.0]\n  [1.0, -8.0, 1.0]\n  [1.0, 1.0, 1.0]'}
